# 04c - Classification Models Development

**Objetivo**: Desarrollar modelos de clasificación para predecir categorías de riesgo de Alzheimer (Low, Moderate, High)
 
**Target Variable**: `risk_category`
**Clases**: Low (46.4%), Moderate (46.1%), High (7.5%)
 
**Modelos a desarrollar**:
- Logistic Regression (baseline)
- Random Forest Classifier
- Gradient Boosting (XGBoost, LightGBM)
- Support Vector Machine

---

## Importar librerías

In [1]:
import sys
import os
sys.path.append('../src/modeling')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import mlflow
import mlflow.sklearn
from sklearn.model_selection import cross_validate
from sklearn.ensemble import StackingClassifier  # Import explícito por si acaso
import warnings
warnings.filterwarnings('ignore')

import model_utils
# Importar scripts personalizados
from classification_pipeline import ClassificationPipeline
from ensemble_methods import AlzheimerEnsemble

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


In [10]:
print("✅ Librerías y scripts importados correctamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


✅ Librerías y scripts importados correctamente
📅 Fecha de ejecución: 2025-06-22 18:18:38


## Cargar datos

In [4]:
# Cargar datos procesados
try:
    df = pd.read_csv('../data/processed/features/alzheimer_features_selected_20250621.csv')
    print(f"📊 Dataset cargado: {df.shape}")
    
    # Verificar target variable
    if 'risk_category' in df.columns:
        print(f"🎯 Distribución de clases:")
        class_dist = df['risk_category'].value_counts()
        print(class_dist)
        print(f"📊 Porcentajes:")
        print((class_dist / len(df) * 100).round(1))
    else:
        print("❌ Error: Variable target 'risk_category' no encontrada")
        
except FileNotFoundError:
    print("❌ Error: Archivo de features no encontrado")
    print("💡 Ejecuta primero el notebook 03_feature_engineering_master.ipynb")


📊 Dataset cargado: (48466, 186)
🎯 Distribución de clases:
risk_category
Low         22501
Moderate    22345
High         3620
Name: count, dtype: int64
📊 Porcentajes:
risk_category
Low         46.4
Moderate    46.1
High         7.5
Name: count, dtype: float64


## Paso 1: Purga de Features con Data Leakage

In [5]:
# =============================================================================
# PASO 1: Purga de features con leakage (VERSIÓN MEJORADA)
# =============================================================================

leakage_features = [
    'risk_category_num',
    'composite_risk_score',
    'diagnosis_code',
    'CDRSB_percentile',
    'CDRSB_LOG',
    'demographic_risk_score',  # Nueva adición
    'multimodal_risk_index'    # Nueva adición
]

# Verificar existencia antes de eliminar
existing_leakage = [f for f in leakage_features if f in df.columns]
df_clean = df.drop(columns=existing_leakage, errors='ignore')

print(f"✅ Features eliminadas: {existing_leakage}")



✅ Features eliminadas: ['composite_risk_score', 'diagnosis_code', 'CDRSB_percentile', 'CDRSB_LOG', 'demographic_risk_score']


## Paso 2: Reingeniería de Features Temporales

In [7]:
# Función mejorada para cambios temporales (sin cambios)
def create_safe_cdrsb_change(df):
    df = df.sort_values(['RID', 'DAYS_SINCE_BASELINE'])
    df['DAYS_SINCE_LAST_VISIT'] = df.groupby('RID')['DAYS_SINCE_BASELINE'].diff()
    df['CDRSB_PREV'] = df.groupby('RID')['CDRSB'].shift(1)
    df['CDRSB_CHANGE_SAFE'] = np.where(
        df['DAYS_SINCE_LAST_VISIT'] >= 90,
        df['CDRSB'] - df['CDRSB_PREV'],
        np.nan
    )
    return df.drop(columns=['CDRSB_PREV', 'DAYS_SINCE_LAST_VISIT'])

df_clean = create_safe_cdrsb_change(df_clean)

# Corregido: Usar columna existente (ej: sleep_efficiency_mean)
df_clean['sleep_quality_trend'] = df_clean.groupby('RID')['sleep_efficiency_mean'].transform(
    lambda x: x.rolling(window=3, min_periods=2).mean()
)

# Imputar columnas
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

imputer = IterativeImputer(max_iter=10, random_state=42)
sleep_cols = ['sleep_minutes_mean', 'sleep_efficiency_mean', 'sleep_disruptions_mean']  # Columnas reales
df_clean[sleep_cols] = imputer.fit_transform(df_clean[sleep_cols])

# Bandera + imputación para CDRSB (sin cambios)
df_clean['CDRSB_MISSING'] = df_clean['CDRSB'].isna().astype(int)
df_clean['CDRSB'] = df_clean.groupby('RID')['CDRSB'].transform(lambda x: x.fillna(x.median()))

## Ejecutar Pipeline

In [8]:
# Inicializar pipeline de clasificación
classification_pipeline = ClassificationPipeline()

# Ejecutar el pipeline completo
print("🚀 Ejecutando pipeline completo de clasificación...")
pipeline_results = classification_pipeline.run_pipeline(df_clean, target_col='risk_category')

# Obtener resultados
results = pipeline_results['results']
cv_results = pipeline_results['cv_results']
trained_models = pipeline_results['trained_models']
best_model_name = pipeline_results['best_model']

print(f"🏆 Mejor modelo seleccionado: {best_model_name}")


🚀 Ejecutando pipeline completo de clasificación...
🚀 Iniciando pipeline de clasificación...
📊 Datos preparados: 38772 train, 9694 test
Entrenando logistic_regression...
Entrenando random_forest...
Entrenando gradient_boosting...
Entrenando svm...
   Usando subconjunto de 10000 muestras para SVM
✅ 4 modelos entrenados
📈 Evaluación completada
Validación cruzada para logistic_regression...
Validación cruzada para random_forest...
Validación cruzada para gradient_boosting...
Validación cruzada para svm...
🔄 Validación cruzada completada
🏆 Mejor modelo: gradient_boosting


2025/07/03 18:43:50 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/03 18:45:27 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


📁 Resultados registrados en MLflow
🏆 Mejor modelo seleccionado: gradient_boosting


## Paso 3: Entrenamiento con Validación Temporal

In [9]:
# Configuración de validación temporal
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, recall_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Ordenar por tiempo desde baseline
df_clean = df_clean.sort_values('DAYS_SINCE_BASELINE')

# Identificar columnas categóricas
categorical_cols = df_clean.select_dtypes(include=['object', 'category']).columns.tolist()
numeric_cols = df_clean.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Remover la columna objetivo de las listas
if 'risk_category' in categorical_cols:
    categorical_cols.remove('risk_category')
if 'risk_category' in numeric_cols:
    numeric_cols.remove('risk_category')

# Crear transformador de columnas
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numeric_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_cols)
    ]
)

# Crear pipeline completo
gbm = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', HistGradientBoostingClassifier(
        loss='log_loss',
        max_depth=4,
        max_iter=150,
        random_state=42,
        class_weight={0: 8.0, 1: 1.0, 2: 3.0}
    ))
])

# Usar RID como identificador de paciente
X = df_clean.drop(columns=['risk_category'])
y = df_clean['risk_category']

# Split temporal basado en tiempo transcurrido
split_index = int(len(df_clean) * 0.8)
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

# Configurar validación cruzada temporal
tscv = TimeSeriesSplit(n_splits=5)

# Bucle de entrenamiento temporal
for train_idx, val_idx in tscv.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_train_fold, y_val_fold = y_train.iloc[train_idx], y_train.iloc[val_idx]
    
    gbm.fit(X_train_fold, y_train_fold)
    y_pred = gbm.predict(X_val_fold)
    
    # Calcular métricas específicas para High Risk
    high_risk_mask = (y_val_fold == 0)
    if sum(high_risk_mask) > 0:
        recall_high_risk = recall_score(
            y_val_fold, 
            y_pred, 
            labels=[0],
            average=None
        )[0]
        print(f"Recall High Risk: {recall_high_risk:.4f}")

# Entrenar modelo final con todos los datos de train
final_model = gbm.fit(X_train, y_train)

# Evaluación en conjunto de prueba temporal
y_test_pred = final_model.predict(X_test)
print("\n🔍 Rendimiento en Test Temporal:")
print(classification_report(y_test, y_test_pred))


🔍 Rendimiento en Test Temporal:
              precision    recall  f1-score   support

        High       1.00      1.00      1.00       343
         Low       1.00      1.00      1.00      6866
    Moderate       1.00      1.00      1.00      2485

    accuracy                           1.00      9694
   macro avg       1.00      1.00      1.00      9694
weighted avg       1.00      1.00      1.00      9694



## Recopilar métricas de rendimiento

In [10]:
# Recopilar métricas de rendimiento
performance_comparison = {}
for name, metrics in results.items():
    performance_comparison[name] = metrics['f1_weighted']

## Ensamblaje de modelos

In [11]:
with mlflow.start_run(run_name="ensemble_classification"):
    mlflow.set_tag("model_family", "ensemble")
    mlflow.set_tag("model_type", "classification")
    
    # 0. Reconstruir datos COMPLETOS desde cero
    X_train, X_test, y_train, y_test = classification_pipeline.prepare_data(df_clean)
    X_full = pd.concat([X_train, X_test])
    y_full = pd.concat([y_train, y_test])
    
    # 1. Definir best_models usando los modelos entrenados
    best_models = {
        'logistic_regression': trained_models['logistic_regression'],
        'random_forest': trained_models['random_forest'],
        'gradient_boosting': trained_models['gradient_boosting'],
        'svm': trained_models['svm']
    }
    
    print("🚀 Creando modelos ensemble...")
    
    # 2. Inicializar ensemble
    ensemble = AlzheimerEnsemble()
    
    # 3. Crear ensembles personalizados
    voting_clf = ensemble.create_custom_voting_ensemble(best_models)
    stacking_clf = ensemble.create_custom_stacking_ensemble(best_models)
    
    # 4. Evaluar con cross_validate (para métricas consistentes)
    from sklearn.model_selection import cross_validate
    
    print(" Evaluando Voting Classifier...")
    voting_cv = cross_validate(
        voting_clf, X_full, y_full,
        cv=3,  # Reducido para mayor velocidad
        scoring=['f1_weighted', 'accuracy'],
        n_jobs=1
    )
    
    print(" Evaluando Stacking Classifier...")
    stacking_cv = cross_validate(
        stacking_clf, X_full, y_full,
        cv=3,  # Reducido para mayor velocidad
        scoring=['f1_weighted', 'accuracy'],
        n_jobs=1
    )
    
    # 5. Entrenar modelos finales (con todos los datos)
    print(" Entrenando modelos finales...")
    voting_clf.fit(X_full, y_full)
    stacking_clf.fit(X_full, y_full)
    
    # 6. Registrar resultados
    mlflow.log_metrics({
        'voting_f1_weighted': voting_cv['test_f1_weighted'].mean(),
        'voting_accuracy': voting_cv['test_accuracy'].mean(),
        'stacking_f1_weighted': stacking_cv['test_f1_weighted'].mean(),
        'stacking_accuracy': stacking_cv['test_accuracy'].mean()
    })
    
    mlflow.sklearn.log_model(voting_clf, "voting_ensemble")
    mlflow.sklearn.log_model(stacking_clf, "stacking_ensemble")
    
    print(f""" Ensembles creados y registrados!
Voting F1: {voting_cv['test_f1_weighted'].mean():.4f}
Stacking F1: {stacking_cv['test_f1_weighted'].mean():.4f}""")

🚀 Creando modelos ensemble...
 Evaluando Voting Classifier...
 Evaluando Stacking Classifier...
 Entrenando modelos finales...


2025/07/03 21:44:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/03 21:46:33 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/07/03 21:46:33 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/07/03 21:46:55 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


 Ensembles creados y registrados!
Voting F1: 0.9979
Stacking F1: 0.9997


4009 segundos de ejecución

In [12]:
# 1. Distribución de clases
print("Distribución de clases:")
print(pd.Series(y_full).value_counts(normalize=True))

# 2. Rendimiento por clase
from sklearn.metrics import classification_report

print("\nReporte Voting Classifier:")
print(classification_report(y_full, voting_clf.predict(X_full)))

print("\nReporte Stacking Classifier:")
print(classification_report(y_full, stacking_clf.predict(X_full)))

Distribución de clases:
risk_category
1    0.464264
2    0.461045
0    0.074692
Name: proportion, dtype: float64

Reporte Voting Classifier:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      3620
           1       1.00      1.00      1.00     22501
           2       1.00      1.00      1.00     22345

    accuracy                           1.00     48466
   macro avg       1.00      1.00      1.00     48466
weighted avg       1.00      1.00      1.00     48466


Reporte Stacking Classifier:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      3620
           1       1.00      1.00      1.00     22501
           2       1.00      1.00      1.00     22345

    accuracy                           1.00     48466
   macro avg       1.00      1.00      1.00     48466
weighted avg       1.00      1.00      1.00     48466



## Paso 4: Evaluación Clínica de Feature Importance

In [15]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report

# 1. Extraer el modelo final del pipeline
if isinstance(final_model, Pipeline):
    model_for_shap = final_model.named_steps[list(final_model.named_steps.keys())[-1]]
    print(f"✅ Modelo extraído del pipeline: {type(model_for_shap).__name__}")
else:
    model_for_shap = final_model

# 2. Configuración segura para baja memoria
SAFE_SAMPLE_SIZE = 30
plt.switch_backend('Agg')  # Backend no interactivo

print(f"🔧 Configuración para baja memoria: {SAFE_SAMPLE_SIZE} muestras")

# 3. Solución definitiva para muestreo con índices alineados
def get_aligned_sample(X, y, sample_size):
    # Crear DataFrame combinado para garantizar alineación
    combined = pd.concat([X, y], axis=1)
    combined.columns = list(X.columns) + ['target']
    
    # Filtrar solo filas completas
    combined = combined.dropna(subset=['target'])
    
    # Muestreo estratificado
    sample = combined.groupby('target', group_keys=False).apply(lambda x: x.sample(min(len(x), max(1, sample_size // len(combined['target'].unique())))))
    
    # Si aún es demasiado grande, tomar muestra aleatoria
    if len(sample) > sample_size:
        sample = sample.sample(sample_size, random_state=42)
    
    # Separar en X e y
    X_sample = sample.drop(columns='target')
    y_sample = sample['target']
    
    return X_sample, y_sample

# 4. Aplicar muestreo seguro con alineación garantizada
try:
    X_test_sample, y_test_sample = get_aligned_sample(X_test, y_test, SAFE_SAMPLE_SIZE)
    print(f"📊 Muestra seleccionada: {len(X_test_sample)} instancias")
    print(f"Distribución de clases: {y_test_sample.value_counts().to_dict()}")
    
except Exception as e:
    print(f"⚠️ Error en muestreo: {str(e)}")
    print("🔁 Usando muestreo simple como alternativa de emergencia")
    
    # Muestreo de emergencia: tomar las primeras n filas
    X_test_sample = X_test.head(SAFE_SAMPLE_SIZE)
    y_test_sample = y_test.head(SAFE_SAMPLE_SIZE)

# 5. Análisis de casos High Risk
clinical_features = [
    'ROCHE_ABETA42_NORMAL',
    'PTAU_pathological',
    'CDRSB',
    'APOE_e4_carrier',
    'age_standardized'
]

if 0 in y_test_sample.values:
    high_risk_mask = (y_test_sample == 0)
    high_risk_sample = X_test_sample.loc[high_risk_mask]
    
    print(f"\n🔍 Casos High Risk en muestra ({len(high_risk_sample)}):")
    
    for i in range(min(3, len(high_risk_sample))):
        row = high_risk_sample.iloc[i]
        print(f"\nCaso {i+1}:")
        print(f"- ABETA42: {row.get('ROCHE_ABETA42_NORMAL', 'N/A')}")
        print(f"- PTAU: {'Patológico' if row.get('PTAU_pathological', 0) == 1 else 'Normal'}")
        print(f"- CDRSB: {row.get('CDRSB', 'N/A')}")
        print(f"- Edad: {row.get('age_standardized', 'N/A'):.2f} SD")
else:
    print("\n⚠️ No se encontraron casos High Risk en la muestra")

# 6. Validación con subpoblaciones clínicas
try:
    if 'ROCHE_ABETA42_NORMAL' in X_test.columns and 'PTAU_pathological' in X_test.columns:
        bio_mask = X_test['ROCHE_ABETA42_NORMAL'].notna() & X_test['PTAU_pathological'].notna()
        
        if bio_mask.sum() > 0:
            y_bio_test = y_test[bio_mask]
            X_bio_test = X_test[bio_mask]
            y_bio_pred = final_model.predict(X_bio_test)
            
            print("\n🧪 Rendimiento en Pacientes con Biomarcadores Completos:")
            print(classification_report(y_bio_test, y_bio_pred))
        else:
            print("\n⚠️ No se encontraron pacientes con biomarcadores completos")
    else:
        print("\n⚠️ Biomarcadores no disponibles para validación")
except Exception as e:
    print(f"⚠️ Error en validación clínica: {str(e)}")

# 7. Análisis de importancia alternativo
print("\n🔝 Método alternativo: Importancia de Características del Modelo")

if hasattr(model_for_shap, 'feature_importances_'):
    # Obtener nombres de características (manejar diferentes estructuras de pipeline)
    if hasattr(X_test, 'columns'):
        feature_names = X_test.columns.tolist()
    else:
        feature_names = [f"feature_{i}" for i in range(X_test.shape[1])]
    
    feat_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model_for_shap.feature_importances_
    }).sort_values('Importance', ascending=False).head(10)
    
    print(feat_importance)
    
    # Guardar resultados
    feat_importance.to_csv('../reports/model_results/model_feature_importance.csv', index=False)
    
    # Visualización simple
    plt.figure(figsize=(10, 6))
    plt.barh(feat_importance['Feature'], feat_importance['Importance'], color='skyblue')
    plt.title('Top 10 Features más Importantes')
    plt.xlabel('Importancia')
    plt.tight_layout()
    plt.savefig('../reports/figures/model_feature_importance.png', dpi=150)
    plt.close()
else:
    print("⚠️ El modelo no tiene atributo 'feature_importances_'")

print("\n✅ Análisis completado con éxito!")

✅ Modelo extraído del pipeline: HistGradientBoostingClassifier
🔧 Configuración para baja memoria: 30 muestras
📊 Muestra seleccionada: 30 instancias
Distribución de clases: {0.0: 10, 1.0: 10, 2.0: 10}

🔍 Casos High Risk en muestra (10):

Caso 1:
- ABETA42: nan
- PTAU: Normal
- CDRSB: nan
- Edad: nan SD

Caso 2:
- ABETA42: 1.0
- PTAU: Normal
- CDRSB: 1.5
- Edad: 0.10 SD

Caso 3:
- ABETA42: nan
- PTAU: Normal
- CDRSB: nan
- Edad: nan SD
⚠️ Error en validación clínica: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

🔝 Método alternativo: Importancia de Características del Modelo
⚠️ El modelo no tiene atributo 'feature_importances_'

✅ Análisis completado con éxito!


In [20]:
import shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import gc  # Para gestión de memoria
import random

# 1. Extraer el modelo final del pipeline
if isinstance(final_model, Pipeline):
    model_for_shap = final_model.named_steps[list(final_model.named_steps.keys())[-1]]
    print(f"✅ Modelo extraído del pipeline: {type(model_for_shap).__name__}")
else:
    model_for_shap = final_model

# 2. Configuración segura para SHAP en baja memoria
SHAP_SAMPLE_SIZE = 5  # Muy pequeño pero significativo
plt.switch_backend('Agg')  # Backend no interactivo
clinical_features = [
    'ROCHE_ABETA42_NORMAL',
    'PTAU_pathological',
    'CDRSB',
    'APOE_e4_carrier',
    'age_standardized'
]

# 3. Función para análisis SHAP seguro
def run_safe_shap_analysis(model, X_sample, features):
    try:
        import shap
        # Liberar memoria antes de SHAP
        gc.collect()
        
        # Crear explainer con configuración ligera
        explainer = shap.TreeExplainer(model, feature_perturbation="interventional")
        
        # Calcular SHAP values solo para las features clínicas
        shap_values = explainer.shap_values(X_sample[features])
        
        # Visualización mínima
        plt.figure(figsize=(10, 6))
        shap.summary_plot(
            shap_values,
            X_sample[features],
            feature_names=features,
            plot_type="dot",
            show=False
        )
        plt.title("Importancia SHAP para Características Clínicas")
        plt.tight_layout()
        plt.savefig('../reports/figures/safe_shap_summary.png', dpi=150)
        plt.close()
        
        print("✅ SHAP ejecutado exitosamente con 5 muestras")
        return True
    except Exception as e:
        print(f"⚠️ SHAP falló: {str(e)}")
        return False

# 4. Seleccionar muestra específica para SHAP
try:
    # Obtener índices de pacientes High Risk
    high_risk_indices = y_test[y_test == 0].index.tolist()
    
    # Seleccionar máximo 5 casos High Risk
    if len(high_risk_indices) > 0:
        shap_sample_indices = random.sample(high_risk_indices, min(SHAP_SAMPLE_SIZE, len(high_risk_indices)))
        X_shap_sample = X_test.loc[shap_sample_indices][clinical_features]
        
        print(f"🔍 Ejecutando SHAP para {len(X_shap_sample)} casos High Risk...")
        shap_success = run_safe_shap_analysis(model_for_shap, X_shap_sample, clinical_features)
        
        if shap_success:
            print("📊 Resultados SHAP guardados en: '../reports/figures/safe_shap_summary.png'")
        else:
            print("🚫 Continuando sin resultados SHAP")
    else:
        print("⚠️ No hay casos High Risk para análisis SHAP")
except Exception as e:
    print(f"⚠️ Error preparando muestra SHAP: {str(e)}")

# 5. Análisis de casos High Risk
clinical_features = [
    'ROCHE_ABETA42_NORMAL',
    'PTAU_pathological',
    'CDRSB',
    'APOE_e4_carrier',
    'age_standardized'
]

if 0 in y_test_sample.values:
    high_risk_mask = (y_test_sample == 0)
    high_risk_sample = X_test_sample.loc[high_risk_mask]
    
    print(f"\n🔍 Casos High Risk en muestra ({len(high_risk_sample)}):")
    
    for i in range(min(3, len(high_risk_sample))):
        row = high_risk_sample.iloc[i]
        print(f"\nCaso {i+1}:")
        print(f"- ABETA42: {row.get('ROCHE_ABETA42_NORMAL', 'N/A')}")
        print(f"- PTAU: {'Patológico' if row.get('PTAU_pathological', 0) == 1 else 'Normal'}")
        print(f"- CDRSB: {row.get('CDRSB', 'N/A')}")
        print(f"- Edad: {row.get('age_standardized', 'N/A'):.2f} SD")
else:
    print("\n⚠️ No se encontraron casos High Risk en la muestra")

# 6. Validación con subpoblaciones clínicas
try:
    if 'ROCHE_ABETA42_NORMAL' in X_test.columns and 'PTAU_pathological' in X_test.columns:
        bio_mask = X_test['ROCHE_ABETA42_NORMAL'].notna() & X_test['PTAU_pathological'].notna()
        
        if bio_mask.sum() > 0:
            y_bio_test = y_test[bio_mask]
            X_bio_test = X_test[bio_mask]
            y_bio_pred = final_model.predict(X_bio_test)
            
            print("\n🧪 Rendimiento en Pacientes con Biomarcadores Completos:")
            print(classification_report(y_bio_test, y_bio_pred))
        else:
            print("\n⚠️ No se encontraron pacientes con biomarcadores completos")
    else:
        print("\n⚠️ Biomarcadores no disponibles para validación")
except Exception as e:
    print(f"⚠️ Error en validación clínica: {str(e)}")

# 7. Análisis de importancia alternativo
print("\n🔝 Método alternativo: Importancia de Características del Modelo")

if hasattr(model_for_shap, 'feature_importances_'):
    # Obtener nombres de características (manejar diferentes estructuras de pipeline)
    if hasattr(X_test, 'columns'):
        feature_names = X_test.columns.tolist()
    else:
        feature_names = [f"feature_{i}" for i in range(X_test.shape[1])]
    
    feat_importance = pd.DataFrame({
        'Feature': feature_names,
        'Importance': model_for_shap.feature_importances_
    }).sort_values('Importance', ascending=False).head(10)
    
    print(feat_importance)
    
    # Guardar resultados
    feat_importance.to_csv('../reports/model_results/model_feature_importance.csv', index=False)
    
    # Visualización simple
    plt.figure(figsize=(10, 6))
    plt.barh(feat_importance['Feature'], feat_importance['Importance'], color='skyblue')
    plt.title('Top 10 Features más Importantes')
    plt.xlabel('Importancia')
    plt.tight_layout()
    plt.savefig('../reports/figures/model_feature_importance.png', dpi=150)
    plt.close()
else:
    print("⚠️ El modelo no tiene atributo 'feature_importances_'")

print("\n✅ Análisis completado con éxito!")

✅ Modelo extraído del pipeline: HistGradientBoostingClassifier
⚠️ Error preparando muestra SHAP: '[6739, 3709, 6737, 3225] not in index'

🔍 Casos High Risk en muestra (10):

Caso 1:
- ABETA42: nan
- PTAU: Normal
- CDRSB: nan
- Edad: nan SD

Caso 2:
- ABETA42: 1.0
- PTAU: Normal
- CDRSB: 1.5
- Edad: 0.10 SD

Caso 3:
- ABETA42: nan
- PTAU: Normal
- CDRSB: nan
- Edad: nan SD
⚠️ Error en validación clínica: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

🔝 Método alternativo: Importancia de Características del Modelo
⚠️ El modelo no tiene atributo 'feature_importances_'

✅ Análisis completado con éxito!


## Comparación Final de Modelos (agregando los ensembles)

In [16]:
print("\n" + "="*60)
print("📊 COMPARACIÓN FINAL DE MODELOS")
print("="*60)

# Crear DataFrame comparativo
comparison_df = pd.DataFrame([
    {'Model': model, 'F1_Score_Weighted': score} 
    for model, score in performance_comparison.items()
]).sort_values('F1_Score_Weighted', ascending=False)


print(comparison_df.to_string(index=False))

# Visualización de comparación
plt.figure(figsize=(12, 6))
bars = plt.bar(comparison_df['Model'], comparison_df['F1_Score_Weighted'], 
               color='lightcoral', alpha=0.8)
plt.title('Comparación de Rendimiento - Modelos de Clasificación')
plt.xlabel('Modelo')
plt.ylabel('F1-Score Weighted')
plt.xticks(rotation=45, ha='right')
plt.grid(axis='y', alpha=0.3)

# Añadir valores en las barras
for bar, score in zip(bars, comparison_df['F1_Score_Weighted']):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.005,
             f'{score:.4f}', ha='center', va='bottom')

plt.tight_layout()
# Guardar imagen
plt.savefig('../reports/figures/model_comparison_classification.png', dpi=300)
plt.show()


📊 COMPARACIÓN FINAL DE MODELOS
              Model  F1_Score_Weighted
  gradient_boosting           0.999794
      random_forest           0.995879
logistic_regression           0.985013
                svm           0.776901


## Resumen final y Recomendaciones

In [18]:
# 1. Definir las variables necesarias si no existen
if 'best_model' not in locals():
    # Obtener el mejor modelo de los resultados del pipeline si existe
    if 'pipeline_results' in locals():
        best_model = pipeline_results.get('best_model', '')
        best_score = pipeline_results['results'].get(best_model, {}).get('f1_weighted', 0)
    else:
        # Calcular el mejor modelo desde performance_comparison
        if 'performance_comparison' in locals():
            best_model = max(performance_comparison.items(), key=lambda x: x[1])[0]
            best_score = performance_comparison[best_model]
        else:
            # Usar el primer modelo de best_models como fallback
            best_model = next(iter(best_models)) if best_models else 'N/A'
            best_score = 0

# 2. Calcular el baseline si no existe
if 'lr_performance' not in locals():
    if 'logistic_regression' in performance_comparison:
        lr_performance = performance_comparison['logistic_regression']
    elif 'logistic_regression' in trained_models:
        from sklearn.metrics import f1_score
        lr_pred = trained_models['logistic_regression'].predict(X_full)
        lr_performance = f1_score(y_full, lr_pred, average='weighted')
    else:
        lr_performance = 0

# 3. Calcular imbalance_ratio si no existe
if 'imbalance_ratio' not in locals():
    class_counts = pd.Series(y_full).value_counts()
    imbalance_ratio = class_counts.max() / class_counts.min()

# 4. Mostrar resumen mejorado
print("\n" + "="*60)
print("🎯 RESUMEN FINAL - MODELOS DE CLASIFICACIÓN")
print("="*60)
print(f"🏆 Mejor modelo: {best_model}")
print(f"📊 F1-Score Weighted: {best_score:.4f}")

if lr_performance > 0:
    improvement = (best_score - lr_performance) / lr_performance * 100
    print(f"📈 Mejora sobre baseline: {improvement:.1f}%")

print(f"⚖️ Desbalanceamiento de clases: {imbalance_ratio:.2f}")
print(f"🧠 Features utilizadas: {len(feature_cols)}")
print(f"🔢 Muestras totales: {len(X_full)}")

# 5. Recomendaciones técnicas
print("\n💡 RECOMENDACIONES BASADAS EN RESULTADOS:")
if best_score > 0.99:
    print("🔍 ¡Resultados excepcionales! Verificar:")
    print("   - Posible data leakage (revisar 'composite_risk_score')")
    print("   - Calidad de los datos (¿valores constantes o duplicados?)")
elif best_score > 0.85:
    print("✅ Excelente rendimiento. Acciones:")
    print("   - Implementar sistema de monitoreo continuo")
    print("   - Documentar importancia de features")
else:
    print("🛠 Oportunidades de mejora:")
    print("   - Optimizar hiperparámetros")
    print("   - Considerar ingeniería de features adicional")

if imbalance_ratio > 3:
    print(f"\n⚖️ ALERTA: Desbalanceo significativo (ratio {imbalance_ratio:.1f}:1)")
    print("   - Técnicas recomendadas:")
    print("     * SMOTE para oversampling")
    print("     * Class weighting en modelos")
    print("     * Métricas adicionales (Precision-Recall Curve)")

# 6. Próximos pasos
print("\n🔜 PRÓXIMOS PASOS RECOMENDADOS:")
print("1. Validación en conjunto de prueba independiente")
print("2. Análisis de errores (matriz de confusión)")
print("3. Revisión clínica de features importantes")
print("4. Documentación técnica completa")


🎯 RESUMEN FINAL - MODELOS DE CLASIFICACIÓN
🏆 Mejor modelo: gradient_boosting
📊 F1-Score Weighted: 0.9995
📈 Mejora sobre baseline: 3.1%
⚖️ Desbalanceamiento de clases: 6.22
🧠 Features utilizadas: 184
🔢 Muestras totales: 48466

💡 RECOMENDACIONES BASADAS EN RESULTADOS:
🔍 ¡Resultados excepcionales! Verificar:
   - Posible data leakage (revisar 'composite_risk_score')
   - Calidad de los datos (¿valores constantes o duplicados?)

⚖️ ALERTA: Desbalanceo significativo (ratio 6.2:1)
   - Técnicas recomendadas:
     * SMOTE para oversampling
     * Class weighting en modelos
     * Métricas adicionales (Precision-Recall Curve)

🔜 PRÓXIMOS PASOS RECOMENDADOS:
1. Validación en conjunto de prueba independiente
2. Análisis de errores (matriz de confusión)
3. Revisión clínica de features importantes
4. Documentación técnica completa


## Guardado de Archivos Importantes

In [18]:
# SOLUCIÓN ROBUSTA PARA GUARDAR MÉTRICAS

# 1. Definir las variables necesarias si no existen
if 'best_model' not in locals():
    # Obtener el mejor modelo de los resultados del pipeline si existe
    if 'pipeline_results' in locals():
        best_model = pipeline_results.get('best_model', '')
        best_score = pipeline_results['results'].get(best_model, {}).get('f1_weighted', 0)
    else:
        # Calcular el mejor modelo desde performance_comparison
        if 'performance_comparison' in locals():
            best_model = max(performance_comparison.items(), key=lambda x: x[1])[0]
            best_score = performance_comparison[best_model]
        else:
            # Usar el primer modelo de best_models como fallback
            best_model = next(iter(trained_models)) if 'trained_models' in locals() else 'N/A'
            best_score = 0

# 2. Calcular el baseline si no existe
if 'lr_performance' not in locals():
    if 'logistic_regression' in performance_comparison:
        lr_performance = performance_comparison['logistic_regression']
    elif 'logistic_regression' in trained_models:
        from sklearn.metrics import f1_score
        lr_pred = trained_models['logistic_regression'].predict(X_full)
        lr_performance = f1_score(y_full, lr_pred, average='weighted')
    else:
        lr_performance = 0

# 3. Calcular imbalance_ratio si no existe
if 'imbalance_ratio' not in locals():
    if 'y_full' in locals():
        class_counts = pd.Series(y_full).value_counts()
    elif 'y_test' in locals():
        class_counts = pd.Series(y_test).value_counts()
    else:
        class_counts = pd.Series(y_train).value_counts()
    
    imbalance_ratio = class_counts.max() / class_counts.min()

# 4. Obtener feature_cols si no existe
if 'feature_cols' not in locals():
    if 'X_train' in locals():
        feature_cols = X_train.columns.tolist()
    elif 'X_full' in locals():
        feature_cols = X_full.columns.tolist()
    else:
        feature_cols = []

# 5. Obtener número de muestras
if 'X_full' in locals():
    training_samples = len(X_full)
elif 'X_train' in locals() and 'X_test' in locals():
    training_samples = len(X_train) + len(X_test)
else:
    training_samples = 0

# 6. Guardar métricas finales
final_metrics = {
    'best_model': best_model,
    'best_f1_score': best_score,
    'baseline_f1_score': lr_performance,
    'improvement_percentage': (best_score - lr_performance) / lr_performance * 100 if lr_performance > 0 else 0,
    'models_trained': len(performance_comparison) if 'performance_comparison' in locals() else 0,
    'imbalance_ratio': imbalance_ratio,
    'training_samples': training_samples,
    'features_used': len(feature_cols),
    'timestamp': pd.Timestamp.now().strftime('%Y-%m-%d %H:%M:%S')
}

# 7. Crear directorio si no existe
import os
os.makedirs('../reports/model_results', exist_ok=True)

# 8. Guardar en archivo
filepath = '../reports/model_results/classification_summary.csv'
pd.Series(final_metrics).to_csv(filepath, header=['Value'], index_label='Metric')

# 9. Mostrar resultados
print("✅ Métricas guardadas en:")
print(f"📂 {os.path.abspath(filepath)}")
print("\nContenido guardado:")
print(pd.Series(final_metrics).to_string())

✅ Métricas guardadas en:
📂 E:\usuarios\alumno\Escritorio\Alzheimer-Multimodal-Monitoring\reports\model_results\classification_summary.csv

Contenido guardado:
best_model                  gradient_boosting
best_f1_score                        0.999794
baseline_f1_score                    0.985013
improvement_percentage               1.500589
models_trained                              4
imbalance_ratio                      6.215746
training_samples                        48466
features_used                             163
timestamp                 2025-07-04 10:24:27


---

__Abraham Tartalos__

---